In [2]:
#pip install torch transformers sentencepiece pandas numpy langchain


In [3]:
#pip install llama-cpp-python


In [4]:
#Running LLaMA 2 Locally (CPU)
#from llama_cpp import Llama

#model_path = "llama_model\llama-2-7b-chat.Q4_K_M.gguf"  # Download a GGUF quantized model
#llm = Llama(model_path=model_path)

In [5]:
import pandas as pd

def excel_to_few_shot_examples(excel_file_path):
    """
    Reads an Excel file and converts it into the specified few-shot examples format.

    Args:
        excel_file_path (str): The path to the Excel file.

    Returns:
        list: A list of dictionaries representing the few-shot examples.
    """
    try:
        df = pd.read_csv(excel_file_path)
    except FileNotFoundError:
        return f"Error: File not found at {excel_file_path}"
    except Exception as e:
        return f"An error occurred: {e}"

    few_shot_examples = []
    for index, row in df.iterrows():
        email = row.get("email")
        request_type = row.get("request_type")
        sub_request_type = row.get("sub_request_type")
        key_attributes_str = row.get("key_attributes")
        main_intent = row.get("main_intent")

        if pd.isna(email) or pd.isna(request_type) or pd.isna(sub_request_type) or pd.isna(main_intent):
            print(f"Warning: Missing data in row {index+1}. Skipping.")
            continue

        key_attributes = []
        if not pd.isna(key_attributes_str):
            key_attributes = [attr.strip() for attr in key_attributes_str.split(",")]

        example = {
            "email": email,
            "expected_output": {
                "request_type": request_type,
                "sub_request_type": sub_request_type,
                "key_attributes": key_attributes,
                "main_intent": main_intent,
            },
        }
        few_shot_examples.append(example)

    return few_shot_examples

# Example usage (replace 'your_excel_file.xlsx' with your actual file path):
result = excel_to_few_shot_examples('Emails.csv')
print(result)

[{'email': 'Sunrise Bank Corporation\n123 Financial Street, New York, NY 10001\nDate: 15-Apr-2025\nATTN: John Doe\nPhone: +1 (212) 555-7890\nFax: +1 (212) 555-7891\nEmail: johndoe@wellsfargo.com\nRe: ABC Holdings LLC – Principal Repayment\nDeal CUSIP: 123456A78\nDeal ISIN: US1234567890\nFacility CUSIP: 987654B32\nFacility ISIN: US0987654321\nLender MEI: LND098765\nEffective 15-Apr-2025, ABC Holdings LLC has elected to make a principal repayment of $250,000 against the loan amount of $5,000,000.\nPrevious Global Principal Balance: $4,750,000\nNew Global Principal Balance: $4,500,000\nThis loan was effective on 01-Jan-2024 and is scheduled to reprice on 01-Jul-2025.\nYour share of the repayment is $250,000.\nWe will remit $250,000 on the effective date. Please note that if:\n(i) the Borrower has not made such payment;\n(ii) any payment you receive is in excess of what was paid by the Borrower; or\n(iii) we notify you that the payment was made in error, you agree to return such payment.\n

In [6]:
def create_few_shot_prompt(email_text):
    few_shot_examples = excel_to_few_shot_examples('Emails.csv')

    few_shot_prompt = "Here are a few examples. Please analyze the provided email text and follow the same pattern as demonstrated in the examples below to generate the output.\n\n"
    for example in few_shot_examples:
        few_shot_prompt += f"Email:{example['email']}Output:{example['expected_output']}\n"

    final_prompt = f"""
{few_shot_prompt}
You are an AI email analyzer for a bank. Categorize the email and extract key details. 
Your output should be a JSON object with the following keys:

- **request_type**: General category (e.g., "Information Request", "Support Request"). 
- **sub_request_type**: Specific category (e.g., "Password Reset", "Order Status"). If not applicable, return "N/A". 
- **key_attributes**: A list of key details (e.g., ["Order Number: 12345", "Account ID: ABC1234"]). If none, return an empty list. 
- **main_intent**: A concise summary of why the email was written. 

Email: 
```{email_text}``` 

Return JSON only, with no extra text before or after.
"""
    return final_prompt

print(create_few_shot_prompt("This is my new input email body"))


Here are a few examples. Please analyze the provided email text and follow the same pattern as demonstrated in the examples below to generate the output.

Email:Sunrise Bank Corporation
123 Financial Street, New York, NY 10001
Date: 15-Apr-2025
ATTN: John Doe
Phone: +1 (212) 555-7890
Fax: +1 (212) 555-7891
Email: johndoe@wellsfargo.com
Re: ABC Holdings LLC – Principal Repayment
Deal CUSIP: 123456A78
Deal ISIN: US1234567890
Facility CUSIP: 987654B32
Facility ISIN: US0987654321
Lender MEI: LND098765
Effective 15-Apr-2025, ABC Holdings LLC has elected to make a principal repayment of $250,000 against the loan amount of $5,000,000.
Previous Global Principal Balance: $4,750,000
New Global Principal Balance: $4,500,000
This loan was effective on 01-Jan-2024 and is scheduled to reprice on 01-Jul-2025.
Your share of the repayment is $250,000.
We will remit $250,000 on the effective date. Please note that if:
(i) the Borrower has not made such payment;
(ii) any payment you receive is in excess

### Old Prompt variable - Markdown not code
prompt = f"""
You are an AI email analyzer for a bank. Categorize the email and extract key details.  
Your output should be a JSON object with the following keys:

- **request_type**: General category (e.g., "Information Request", "Support Request").  
- **sub_request_type**: Specific category (e.g., "Password Reset", "Order Status"). If not applicable, return "N/A".  
- **key_attributes**: A list of key details (e.g., ["Order Number: 12345", "Account ID: ABC1234"]). If none, return an empty list.  
- **main_intent**: A concise summary of why the email was written.  

Email:  
```{email_text}```  

Return JSON only, with no extra text before or after.
"""

In [ ]:
import json
import re
from llama_cpp import Llama

def categorize_email(email_text, model_path):
    """
    Uses Llama 2 (locally) to categorize an email and extract key details.

    Args:
        email_text: The text content of the email.
        model_path: The path to the locally stored LLaMA 2 model.

    Returns:
        A JSON-formatted string with the categorized information.
    """
    ## ENABLE THSI FEW SHOTS
    #prompt = create_few_shot_prompt(email_text)

    prompt = f"""
    You are an AI email analyzer for a bank. Categorize the email and extract key details.  
    Your output should be a JSON object with the following keys:

    - **request_type**: General category (e.g., "Information Request", "Support Request").  
    - **sub_request_type**: Specific category (e.g., "Password Reset", "Order Status"). If not applicable, return "N/A".  
    - **key_attributes**: A list of key details (e.g., ["Order Number: 12345", "Account ID: ABC1234"]). If none, return an empty list.  
    - **main_intent**: A concise summary of why the email was written.  

    Email:  
    ```{email_text}```  

    Return JSON only, with no extra text before or after.
    """
    # Load LLaMA 2 locally

    ## ENABLE THSI FEW SHOTS
    #llm = Llama(model_path=model_path, n_ctx=4096)
    llm = Llama(model_path=model_path)
    # Generate response
    response = llm(prompt, max_tokens=5000)

    # Extract the text output
    response_text = response["choices"][0]["text"].strip()

    # Extract JSON using regex
    json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
    
    if json_match:
        try:
            result = json.loads(json_match.group(0))
            return json.dumps(result, indent=4)
        except json.JSONDecodeError:
            return json.dumps({"error": "Failed to parse JSON from the model response."}, indent=4)
    
    return json.dumps({"error": "No JSON response detected from the model."}, indent=4)




In [13]:
email_samples = [
    # 1️⃣ Account Issue - Unrecognized Transaction
    """Subject: Email 5: Credit Limit Increase Notification
[Sunrise Bank Corporation]
123 Finance Street, New York, NY 10001
Date: 05-Apr-2025
ATTN: Kevin Harris
Phone: (555) 567-8901
Fax: (555) 543-2109
Email: kevin.harris@sunrisebank.com
Re: Credit Limit Increase Notification
Effective 05-Apr-2025, the credit limit for your loan has been updated as follows:
Previous Commitment Amount: $10,000,000
New Commitment Amount: $12,500,000
Please update your records accordingly. Should you have any questions or require further clarification, please do not hesitate to contact us.
Thanks & Regards,
Brian Carter
Account Manager
Sunrise Bank Corporation
brian.carter@sunrisebank.com""" 
]

# Example usage with all sample emails:
model_path = "llama_model/llama-2-7b-chat.Q4_K_M.gguf"

for email in email_samples:
    print(categorize_email(email, model_path))
    print("-" * 80)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama_model/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32    

{
    "error": "No JSON response detected from the model."
}
--------------------------------------------------------------------------------
